In [ ]:
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline



# Váriaveis Repetidas

In [ ]:
random_state = 42
alpha = 0.05
test_size = 0.3

# 4-3-3.ipynb

# Obtenha a média e o desvio padrão da Accuracy; Sensitivity; Specificity e F1 do atributo RespDisease com os modelos obtidos na alínea anterior. 

# Leitura de Dados e filtração de Dados de Alinea anterior.

In [ ]:
# comando read_csv para tal.
dados = pd.read_csv('../../dados/AIRPOL_data.csv', delimiter=";", header=0, decimal=',')
dados = dados.drop(columns=['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'])


goalAttrib = 'RespDisease'

def calc_resp_disease(row):
    respDiseases = ['Asthma', 'Chronic obstructive pulmonary disease']
    if row['Outcome'] in respDiseases:  # Use 'Outcome' instead of 'Disease'
        return 1
    else:
        return 0

dados[goalAttrib] = dados.apply(calc_resp_disease, axis=1)
dados 


# K- FOLD Cross Validation (linha anterior)

In [ ]:
features = list(dados.columns[0:9])
numericFeatures = features[4:]
print(numericFeatures)
scaler = StandardScaler()

X = dados[numericFeatures].drop(columns=[goalAttrib])
y = dados[goalAttrib]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=test_size,random_state=random_state)

print("Stratified division of goal attribute:")
print(y_train.value_counts(normalize=True).mul(100).round(1).astype(str)+'%')
print(y_test.value_counts(normalize=True).mul(100).round(1).astype(str)+'%')

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Definição dos Modelos K-NEIGHBOURS,SVM,REGRESSION THREE,NEURAL NETWORK With ITS CLASSIFIERES com cada um dos seus construtores.

In [ ]:
models = []
models.append(('dtr', DecisionTreeClassifier(  # Decision Tree Classifier
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=2,
    random_state=random_state
)))

models.append(('net', MLPClassifier(
    hidden_layer_sizes=(5, 6),
    activation='tanh',
    solver='lbfgs',
    max_iter=1000,
    learning_rate='adaptive',
    early_stopping=True,
    random_state=random_state
)))

models.append(('knn', KNeighborsClassifier(n_neighbors=21)))
models.append(('svm', make_pipeline(StandardScaler(), SVC(kernel='rbf'))))

# Definição das Varaiáveis  [Accuracy, Sensitivity, Specificity, F1]

In [ ]:

from sklearn.metrics import make_scorer, accuracy_score, recall_score, confusion_matrix, f1_score

# Funções para Sensitivity (recall) e Specificity
def sensitivity(y_true, y_pred):
    return recall_score(y_true, y_pred, pos_label=1)

def specificity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn = cm[0, 0]
    fp = cm[0, 1]
    return tn / (tn + fp)
    
# Preparar dados (imputação de NaN)
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

#Scorer personalizados
scorers = {
    'accuracy': make_scorer(accuracy_score),
    'sensitivity': make_scorer(sensitivity),
    'specificity': make_scorer(specificity),
    'f1': make_scorer(f1_score)
}



# Cross Validation para accucary , sensitivity, specificity e f1 usando os modelos definidos.

In [ ]:

# Initialize results dictionary
results = {name: {metric: [] for metric in scorers} for name, _ in models}

# Método usado para calcular as métricas
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

for name, model in models:
    print(f'\nTreinando modelo: {name}')
    start_time = time.time()
    
    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_imputed, y)):
        fold_start = time.time()
        
        X_train, X_test = X_imputed.iloc[train_idx], X_imputed.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        results[name]['accuracy'].append(accuracy_score(y_test, y_pred))
        results[name]['sensitivity'].append(sensitivity(y_test, y_pred))
        results[name]['specificity'].append(specificity(y_test, y_pred))
        results[name]['f1'].append(f1_score(y_test, y_pred))
        
        fold_end = time.time()
        print(f'  Fold {fold_idx + 1} em {fold_end - fold_start:.2f} segundos')

    end_time = time.time()
    print(f'Tempo total para {name}: {end_time - start_time:.2f} segundos')

summary = []

for model_name, metrics in results.items():
    row = {'Model': model_name}
    for metric_name, values in metrics.items():
        row[f'{metric_name}_mean'] = np.mean(values)
        row[f'{metric_name}_std'] = np.std(values)
    summary.append(row)

df_summary = pd.DataFrame(summary)
df_summary


# 4-3-4  Verifique  se  existe  diferença  significativa  no  desempenho  dos  dois  melhores modelos obtidos anteriormente (use um nível de significância de 5%). Identifique o modelo que apresenta o melhor desempenho

#  (Hold out 80% 20%)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Divide os dados em 80% treino e 20% teste
X_train, X_test, y_train, y_test = train_test_split(
    X_imputed, y, test_size=0.20, stratify=y, random_state=42
)

### 2 Melhores modelos Escolhidos

1. **Decision Tree (DTR)**  
2. **K-Nearest Neighbors (KNN)**

## Explicação 
## Melhor modelo DTR
- **Maior F1-score** (`0.6577`), mostra excelente equilíbrio entre precisão e a sensibilidade.
- **Maior sensibilidade** (`0.5124`), ou seja, o modelo consegue identificar corretamente mais casos positivos.
- **Melhor acuracy** (`0.7966`)  
- **specificity** (`0.9718`), o que confirma um desempenho forte e estável para além de isso tem um **baixo desvio padrão** (`0.0071`), indicando que o modelo é consistente em diferentes folds da validação cruzada.


## Segundo melhor modelo:KNN
- Foi o segundo melhor em **F1-score** (`0.4021`) e em **sensibilidade** (`0.2585`).
- Apesar de inferior ao DTR, teve um desempenho consideravelmente melhor do que os modelos SVM e Net, que falharam em identificar casos positivos.
- Também apresentou **specificity alta** (`0.9831`) e **boa estabilidade** nos resultados.


# Uso de Decesion Tree para obter o melhor modelo


Treinando modelo: dtr
  Fold 1 em 0.22 segundos
  Fold 2 em 0.27 segundos
  Fold 3 em 0.16 segundos
  Fold 4 em 0.30 segundos
  Fold 5 em 0.22 segundos
Tempo total para dtr: 1.20 segundos

Treinando modelo: net
  Fold 1 em 0.58 segundos
  Fold 2 em 0.49 segundos
  Fold 3 em 0.27 segundos
  Fold 4 em 0.38 segundos
  Fold 5 em 0.46 segundos
Tempo total para net: 2.21 segundos

Treinando modelo: knn
  Fold 1 em 1.68 segundos
  Fold 2 em 1.38 segundos
  Fold 3 em 1.47 segundos
  Fold 4 em 1.07 segundos
  Fold 5 em 1.05 segundos
Tempo total para knn: 6.67 segundos

Treinando modelo: svm


In [ ]:
# Supondo que os dois melhores modelos sejam 'dtr' e 'knn'
from scipy.stats import ttest_rel

# Obter os F1-scores de cada modelo nos 5 folds da cross-validation
f1_dtr = results['dtr']['F1']['Valores']  # Lista com os F1 do modelo DTR
f1_knn = results['knn']['F1']['Valores']  # Lista com os F1 do modelo KNN

# Aplicar teste t pareado
stat, p_value = ttest_rel(f1_dtr, f1_knn)

print("Resultado do teste t para comparação dos modelos DTR e KNN:")
print(f"Estatística t: {stat:.4f}")
print(f"Valor-p: {p_value:.4f}")

# Conclusão baseada no valor-p
if p_value < 0.05:
    print("\nExiste diferença estatisticamente significativa entre os dois modelos.")
    melhor = 'dtr' if np.mean(f1_dtr) > np.mean(f1_knn) else 'knn'
    print(f" O modelo com melhor desempenho médio é: {melhor.upper()}")
else:
    print("\n Não existe diferença estatisticamente significativa entre os modelos.")


# 4-3-5 Compare os resultados dos modelos. Discuta em detalhe qual o modelo que apresentou melhor e pior desempenho de acordo com os critérios: Accuracy; Sensitivity; Specificity e F1.  



#### Melhor Desempenho (DTR)
  **Accuracy:** 0.7966 → maior taxa geral de acertos.
  **Sensitivity:** 0.5124 → o único modelo que conseguiu identificar corretamente mais de 50% dos casos opositivos.
  **Specificity:** 0.9718 → bom desempenho na identificação dos negativos.
  **F1-score:** 0.6577 → o mais alto entre todos, indicando equilíbrio entre precisão e sensibilidade.
  
  **Conclusão 1:** O DTR é o **modelo mais completo e equilibrado**, com desempenho superior em todas as métricas mais relevantes.


#### Pior Desempenho (Net)
  **Accuracy:** 0.6187 → desempenho fraco no geral.
  **Sensitivity:** 0.0000 → nenhum caso positivo.
  **Specificity:** 1.0000 → classificou todos os negativos corretamente, mas ignorou completamente os positivos.
  **F1-score:** 0.0000 → desempenho inaceitável para problemas que exigem detecção de positivos.
  **Conclusão 2:** Apesar da especificidade perfeita, o modelo é **inútil para detectar casos relevantes**.


###  Conclusão Geral
O **modelo com melhor desempenho global** é o **Decision Tree (DTR)**, sendo o único com boas métricas em todos os critérios.
O **pior modelo** foi o **Net**, que falhou completamente na detecção da classe positiva, mesmo apresentando accuracy e specificity aparentemente aceitáveis.
